In [5]:
import os
import numpy as np
import torch
import librosa
import json
from jiwer import wer
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from pyctcdecode import build_ctcdecoder

def preprocess_audio(file_path):
    """Tiền xử lý file âm thanh thành dạng phù hợp với mô hình."""
    try:
        # Kiểm tra file tồn tại
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"Không tìm thấy file âm thanh: {file_path}")
        
        # Đọc và chuyển đổi về tần số lấy mẫu 16kHz
        audio, orig_sr = librosa.load(file_path, sr=None)
        if orig_sr != 16000:
            # print(f"Chuyển đổi từ {orig_sr}Hz sang 16kHz...")
            audio = librosa.resample(audio, orig_sr=orig_sr, target_sr=16000)
            
        # # Cắt nếu độ dài quá 10 giây
        # if len(audio) > 160000:  # 10 giây ở 16kHz
        #     print("Cảnh báo: Âm thanh dài hơn 10 giây. Đang cắt...")
        #     audio = audio[:160000]
            
        return audio
        
    except Exception as e:
        raise RuntimeError(f"Lỗi khi xử lý file âm thanh: {e}")

def transcribe_audio(file_path):
    """Chuyển đổi âm thanh thành văn bản."""
    try:
        # Tiền xử lý âm thanh
        audio = preprocess_audio(file_path)
        
        # Tokenize với sampling_rate được chỉ định rõ ràng
        input_values = processor(audio, return_tensors="pt", padding="longest", sampling_rate=16000).input_values
        
        # Lấy logits từ mô hình
        with torch.no_grad():
            logits = model(input_values).logits.cpu().numpy()[0]
        
        # Giải mã
        if use_lm:
            # Sử dụng beam search với language model
            beam_results = decoder.decode_beams(logits, beam_width=100)
            transcription = beam_results[0][0]
        else:
            # Sử dụng argmax decoding cơ bản
            predicted_ids = torch.argmax(torch.tensor(logits), dim=-1)
            transcription = processor.decode(predicted_ids)
        
        return transcription
        
    except Exception as e:
        print(f"Lỗi khi chuyển đổi âm thanh: {e}")
        return None

def convert(file_path):
    """Chuyển đổi một file âm thanh và in kết quả."""
    result = transcribe_audio(file_path)
    if result:
        print(f"\nKết quả chuyển đổi ({os.path.basename(file_path)}):")
        print(result)
    return result

def evaluate_wer(reference_text, hypothesis_text):
    """Tính toán Word Error Rate giữa văn bản tham chiếu và văn bản nhận dạng."""
    if not reference_text or not hypothesis_text:
        return 1.0  # Trả về lỗi 100% nếu một trong hai chuỗi trống
    
    # Tính toán WER
    error_rate = wer(reference_text, hypothesis_text)
    return error_rate

def read_reference(reference_file):
    """Đọc văn bản tham chiếu từ file."""
    try:
        with open(reference_file, 'r', encoding='utf-8') as f:
            return f.read().strip()
    except Exception as e:
        print(f"Không thể đọc file tham chiếu {reference_file}: {e}")
        return None

def convert_list(audio_dir, reference_dir):
    """
    Xử lý tất cả các file âm thanh trong thư mục và đánh giá với file tham chiếu.
    
    Parameters:
    audio_dir (str): Đường dẫn đến thư mục chứa file âm thanh
    reference_dir (str): Đường dẫn đến thư mục chứa file tham chiếu
    
    Returns:
    dict: Kết quả theo định dạng {file_name: {transcription: str, reference: str, wer: float}}
    """
    results = {}
    total_wer = 0
    count = 0
    
    # Kiểm tra thư mục tồn tại
    if not os.path.isdir(audio_dir):
        print(f"Thư mục âm thanh không tồn tại: {audio_dir}")
        return results
    
    if not os.path.isdir(reference_dir):
        print(f"Thư mục tham chiếu không tồn tại: {reference_dir}")
        return results
    
    # Định dạng file âm thanh phổ biến
    audio_extensions = ['.wav', '.mp3', '.flac', '.m4a', '.aac', '.ogg']
    
    # Lấy danh sách file âm thanh
    audio_files = []
    for file_name in os.listdir(audio_dir):
        _, ext = os.path.splitext(file_name)
        if ext.lower() in audio_extensions:
            audio_files.append(file_name)
    
    print(f"Tìm thấy {len(audio_files)} file âm thanh. Đang bắt đầu xử lý...")
    
    # Xử lý từng file
    for file_name in audio_files:
        # Đường dẫn đầy đủ đến file âm thanh
        audio_path = os.path.join(audio_dir, file_name)
        
        # Tìm file tham chiếu tương ứng (giả sử có cùng tên file, đuôi .txt)
        base_name = os.path.splitext(file_name)[0]
        reference_path = os.path.join(reference_dir, f"{base_name}.txt")
        
        # Kiểm tra xem file tham chiếu có tồn tại không
        if not os.path.exists(reference_path):
            print(f"Không tìm thấy file tham chiếu {reference_path} cho {file_name}")
            continue
        
        # Chuyển đổi âm thanh thành văn bản
        transcription = transcribe_audio(audio_path)
        if not transcription:
            print(f"Không thể chuyển đổi file {file_name}")
            continue
        
        # Đọc văn bản tham chiếu
        reference_text = read_reference(reference_path)
        if not reference_text:
            continue
        
        # Tính toán WER
        error_rate = evaluate_wer(reference_text, transcription)
        total_wer += error_rate
        count += 1
        
        # Lưu kết quả
        results[file_name] = {
            "transcription": transcription,
            "reference": reference_text,
            "wer": error_rate
        }
        
        print(f"File: {file_name}")
        print(f"Tham chiếu: {reference_text}")
        print(f"Nhận dạng: {transcription}")
        print(f"WER: {error_rate:.4f} ({error_rate*100:.2f}%)")
        print("-" * 50)
    
    # Tính WER trung bình
    if count > 0:
        avg_wer = total_wer / count
        print(f"WER trung bình: {avg_wer:.4f} ({avg_wer*100:.2f}%)")
    
    # Lưu kết quả vào file JSON
    with open(audio_dir + '_evaluation_results.json', 'w', encoding='utf-8') as f:
        json.dump(results, f, ensure_ascii=False, indent=4)
    
    print(f"Đã lưu kết quả vào file evaluation_results.json")
    
    return results

# **Loading model**

In [6]:
# Tải mô hình và processor (giả sử đã được định nghĩa trước đó)
model_name = "nguyenvulebinh/wav2vec2-base-vietnamese-250h"
processor = Wav2Vec2Processor.from_pretrained(model_name)
model = Wav2Vec2ForCTC.from_pretrained(model_name)

# Thiết lập CTC decoder
vocab_dict = processor.tokenizer.get_vocab()
sort_vocab = sorted((value, key) for (key, value) in vocab_dict.items())
vocab = [x[1] for x in sort_vocab]

# Kiểm tra mô hình ngôn ngữ
lm_file = "vi_lm_4grams.bin"
use_lm = False

if os.path.exists(lm_file):
    try:
        import kenlm
        decoder = build_ctcdecoder(
            vocab,
            kenlm_model_path=lm_file,
            alpha=0.5,
            beta=1.0,
        )
        use_lm = True
        print("Đã tải thành công mô hình ngôn ngữ!")
    except Exception as e:
        print(f"Không thể tải mô hình ngôn ngữ: {e}")
        decoder = build_ctcdecoder(vocab)
else:
    print("Không tìm thấy mô hình ngôn ngữ, sử dụng giải mã cơ bản")
    decoder = build_ctcdecoder(vocab)

/home/ngocducpc/miniconda3/envs/speech/lib/python3.12/site-packages/transformers/configuration_utils.py:311: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Loading the LM will be faster if you build a binary file.
Reading /home/ngocducpc/Research/src/CTC-SpeechRefinement/vi_lm_4grams.bin
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Found entries of length > 1 in alphabet. This is unusual unless style is BPE, but the alphabet was not recognized as BPE type. Is this correct?


Không thể tải mô hình ngôn ngữ: Cannot read model 'vi_lm_4grams.bin' (lm/read_arpa.cc:65 in void lm::ReadARPACounts(util::FilePiece&, std::vector<long unsigned int>&) threw FormatLoadException. first non-empty line was "Entry not found" not \data\. Byte: 15)


# **Part I**

In [7]:
audio_dir = "data/audio"
reference_dir = "data/transcripts"
convert_list(audio_dir, reference_dir)

Tìm thấy 6 file âm thanh. Đang bắt đầu xử lý...
File: test1_01.wav
Tham chiếu: Sau 50 năm thống nhất, Thành phố Hồ Chí Minh đã vươn mình trở thành một đô thị hiện đại, sôi động bậc nhất Đông Nam Á, với những tòa cao ốc chọc trời như Saigon Centre, Bitexco Financial Tower, Landmark 81 và sắp tới là Empire 88 Tower ở Thủ Thiêm.
Nhận dạng: sau năm mươi năm thống nhất thành phố hồ chí minh đã vươn mình thành đô thị hiện đại sôi động bậc nhất đông nam á với những tòa cao ốc trọc trời như sài gòn sen trờ bi tet cô phai nen sô thao ở len mác tám mươi mốt và sắp tới là em pai tám mươi tám thao ở ở thủ thiêm
WER: 0.7885 (78.85%)
--------------------------------------------------
File: test1_06.wav
Tham chiếu: Tìm các nhà hàng Ý có đánh giá trên 4 sao trong bán kính 5 km.
Nhận dạng: tìm kiếm các nhà hàng ý có đánh giá trên bốn sao trong bán kính năm ki lô mét
WER: 0.5000 (50.00%)
--------------------------------------------------
File: test1_03.wav
Tham chiếu: Gửi email cho toàn bộ nhóm dự án về

{'test1_01.wav': {'transcription': 'sau năm mươi năm thống nhất thành phố hồ chí minh đã vươn mình thành đô thị hiện đại sôi động bậc nhất đông nam á với những tòa cao ốc trọc trời như sài gòn sen trờ bi tet cô phai nen sô thao ở len mác tám mươi mốt và sắp tới là em pai tám mươi tám thao ở ở thủ thiêm',
  'reference': 'Sau 50 năm thống nhất, Thành phố Hồ Chí Minh đã vươn mình trở thành một đô thị hiện đại, sôi động bậc nhất Đông Nam Á, với những tòa cao ốc chọc trời như Saigon Centre, Bitexco Financial Tower, Landmark 81 và sắp tới là Empire 88 Tower ở Thủ Thiêm.',
  'wer': 0.7884615384615384},
 'test1_06.wav': {'transcription': 'tìm kiếm các nhà hàng ý có đánh giá trên bốn sao trong bán kính năm ki lô mét',
  'reference': 'Tìm các nhà hàng Ý có đánh giá trên 4 sao trong bán kính 5 km.',
  'wer': 0.5},
 'test1_03.wav': {'transcription': 'gửi i meu cho toàn bộ nhóm dự án về việc hoãn cuộc học sáng mai sang chiều thứ sáu và nhắc họ chuẩn bị báo cáo tiến độ',
  'reference': 'Gửi email ch

# **Part II**

In [8]:
audio_dir = "data/errordetect"
reference_dir = "data/transcripts"
convert_list(audio_dir, reference_dir)

Tìm thấy 10 file âm thanh. Đang bắt đầu xử lý...
File: test2_08.wav
Tham chiếu: đặt meeting online với team lúc trưa
Nhận dạng: đặt mít tinh on lai với tim lúc chưa
WER: 0.8571 (85.71%)
--------------------------------------------------
File: test2_05.wav
Tham chiếu: Thời tiết Hà Nội ngày mai như thế nào
Nhận dạng: thới tiết hà nội ngày mai thế nào
WER: 0.4444 (44.44%)
--------------------------------------------------
File: test2_03.wav
Tham chiếu: Chỉ đường đến khách sạn Marriott trung tâm
Nhận dạng: chỉ đường đến khách sạn ma ri ớt trung tâm
WER: 0.5000 (50.00%)
--------------------------------------------------
File: test2_06.wav
Tham chiếu: tạo nhắc nhở mua sữa khi về nhà
Nhận dạng: tạo nhắc nhở mua sữa khi về nhà
WER: 0.0000 (0.00%)
--------------------------------------------------
File: test2_04.wav
Tham chiếu: Tìm kiếm sách của tác giả Stephen King
Nhận dạng: tìm kiếm sách của tác giả ti phần kinh
WER: 0.5000 (50.00%)
--------------------------------------------------
File: te

{'test2_08.wav': {'transcription': 'đặt mít tinh on lai với tim lúc chưa',
  'reference': 'đặt meeting online với team lúc trưa',
  'wer': 0.8571428571428571},
 'test2_05.wav': {'transcription': 'thới tiết hà nội ngày mai thế nào',
  'reference': 'Thời tiết Hà Nội ngày mai như thế nào',
  'wer': 0.4444444444444444},
 'test2_03.wav': {'transcription': 'chỉ đường đến khách sạn ma ri ớt trung tâm',
  'reference': 'Chỉ đường đến khách sạn Marriott trung tâm',
  'wer': 0.5},
 'test2_06.wav': {'transcription': 'tạo nhắc nhở mua sữa khi về nhà',
  'reference': 'tạo nhắc nhở mua sữa khi về nhà',
  'wer': 0.0},
 'test2_04.wav': {'transcription': 'tìm kiếm sách của tác giả ti phần kinh',
  'reference': 'Tìm kiếm sách của tác giả Stephen King',
  'wer': 0.5},
 'test2_09.wav': {'transcription': 'tim mác phôn giá tốt dưới năm triệu',
  'reference': 'tìm smartphone giá tốt dưới năm triệu',
  'wer': 0.42857142857142855},
 'test2_10.wav': {'transcription': 'múc cho tôi vé máy bay đi sin ga po ngày mai